### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy import plotting
import subjects

### Detect micro-arousals

In [ ]:
from neuropy.core import Epoch

sessions = subjects.ripple_sess()
# sessions = subjects.sd.ratVday2

for s, sess in enumerate(sessions):
    states_df = sess.brainstates_fine.to_dataframe()
    states_df.loc[
        (states_df.label == "AW") | (states_df.label == "QW"), ["label"]
    ] = "WK"
    # states_df.loc[
    #     (states_df.label == "REM") | (states_df.label == "NREM"), ["label"]
    # ] = "SLP"

    states = Epoch(states_df).merge_neighbors()
    labels = states.labels
    durations = states.durations
    starts, stops = states.starts, states.stops

    ma_epochs = []
    for i in range(1, len(states) - 1):
        cond0 = (labels[i] == "WK") & (durations[i] <= 100)
        cond1 = (labels[i - 1] == "NREM") & (labels[i + 1] == "NREM")
        cond2 = (labels[i - 1] == "NREM") & (labels[i + 1] == "REM")
        cond3 = (labels[i - 1] == "REM") & (labels[i + 1] == "NREM")
        cond4 = (labels[i - 1] == "REM") & (labels[i + 1] == "REM")

        if cond0 & cond1:
            ma_epochs.append([starts[i], stops[i], "MA_NREM"])

        if cond0 & (cond2 | cond3 | cond4):
            ma_epochs.append([starts[i], stops[i], "MA_REM"])
    ma_epochs = np.array(ma_epochs)
    micro_wake = Epoch.from_array(
        ma_epochs[:, 0].astype("float"),
        ma_epochs[:, 1].astype("float"),
        ma_epochs[:, 2].astype("str"),
    )
    micro_wake.save(sess.filePrefix.with_suffix(".micro_arousals.npy"))

### Rate of micro_arousals SD vs NSD

In [ ]:
sessions = subjects.ripple_sess()

ma_df = []
for sub, sess in enumerate(sessions):
    epochs = sess.get_zt_epochs()[2:]
    ma = sess.micro_arousals.starts

    hist_ma = np.histogram(ma, bins=epochs.flatten())[0][::2]

    ma_df.append(
        pd.DataFrame(
            {
                "number": hist_ma,
                "zt": epochs.labels,
                "session": sub,
                "name": sess.name[:4],
                "grp": sess.tag,
            }
        ),
    )

ma_df = pd.concat(ma_df, ignore_index=True)

# subjects.GroupData().save(ma_df, "micro_arousal_number")

In [ ]:
# _, ax = plt.subplots()
fig = plotting.Fig(6, 4)
ax = fig.subplot(fig.gs[0])

sns.stripplot(
    data=ma_df,
    x="zt",
    y="number",
    hue="grp",
    dodge=True,
    palette=subjects.colors_sd(1),
    ax=ax,
)
ax.legend("", frameon=False)
fig.savefig(subjects.figpath_sd / "micro_arousal_rate")

### Duration comparison

In [ ]:
sessions = subjects.ripple_sess()

ma_df = []
for sub, sess in enumerate(sessions):
    epochs = sess.get_zt_epochs()[2:]
    ma = sess.micro_arousals["MA_REM"]

    for e in epochs.itertuples():
        e_ma = ma.time_slice(e.start, e.stop, strict=False)
        df = pd.DataFrame(
            {
                "duration": e_ma.durations,
                "zt": e.label,
                "session": sub,
                "name": sess.name[:4],
                "grp": sess.tag,
            }
        )

        ma_df.append(df)

ma_df = pd.concat(ma_df, ignore_index=True)

In [ ]:
fig = plotting.Fig(6, 4)
ax = fig.subplot(fig.gs[0])
sns.violinplot(
    data=ma_df, x="zt", y="duration", hue="grp", palette=subjects.colors_sd(1), ax=ax
)

fig.savefig(subjects.figpath_sd / "micro_arousal_duration")